# Classificação de Respostas de Analistas com BERTimbau
Este notebook mostra, passo a passo, como treinar um classificador binário (0 = resposta **incompleta**, 1 = resposta **completa**) usando o modelo **BERTimbau** da Neuralmind e a biblioteca 🤗 *Transformers*.

## 0. Pré‑requisitos
Instalamos as principais bibliotecas necessárias para todo o fluxo:

In [ ]:
!pip install -q transformers datasets evaluate torch scikit-learn

## 1. Carregar e inspecionar o conjunto de dados
Leitura do CSV com as colunas **`texto_manifestacao`**, **`texto_resposta`** e **`analise_resposta`**. Convertendo o rótulo para inteiro e conferindo o balanceamento das classes.

In [ ]:
import pandas as pd

# 🔁 Altere o caminho do arquivo se necessário
df = pd.read_csv("manifestacoes.csv")
df["analise_resposta"] = df["analise_resposta"].astype(int)

display(df['analise_resposta'].value_counts())
df.head()

## 2. Fundir manifestação + resposta (opcional)
Em muitos casos apenas a **resposta** já basta, mas podemos concatenar a manifestação para dar contexto. A separação `[SEP]` ajuda o modelo a distinguir as duas partes.

In [ ]:
df['texto_full'] = df['texto_manifestacao'].fillna('') + ' [SEP] ' + df['texto_resposta'].fillna('')

## 3. Criar `Dataset` HuggingFace e dividir em treino/validação
Usamos *train_test_split* estratificado para manter a proporção de classes.

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df, test_size=0.15, stratify=df['analise_resposta'], random_state=42
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))

## 4. Tokenizar com o BERTimbau
Carregamos o **tokenizer** e aplicamos truncamento e padding para tamanho fixo de sequência.

In [ ]:
from transformers import AutoTokenizer

checkpoint = 'neuralmind/bert-base-portuguese-cased'
tokenizer  = AutoTokenizer.from_pretrained(checkpoint)

def tokenize(batch):
    return tokenizer(
        batch['texto_full'],
        truncation=True,
        padding='max_length',
        max_length=256
    )

train_ds = train_ds.map(tokenize, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(tokenize, batched=True, remove_columns=val_ds.column_names)

train_ds.set_format('torch')
val_ds.set_format('torch')

## 5. Definir o modelo de classificação
`BertForSequenceClassification` adiciona uma camada linear ao topo do BERT. `num_labels=2` informa que o problema é binário.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2
)

## 6. Configurar treinamento
Definimos hiperparâmetros no `TrainingArguments` e passamos tudo ao `Trainer`. Métricas usadas: *accuracy* e *F1‑macro*. Ajuste `per_device_train_batch_size` conforme sua GPU/CPU.

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate, numpy as np

accuracy = evaluate.load('accuracy')
f1       = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy.compute(predictions=preds, references=labels)['accuracy'],
        'f1':       f1.compute(predictions=preds, references=labels, average='macro')['f1']
    }

args = TrainingArguments(
    output_dir='bertimbau-resposta',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    greater_is_better=True,
    logging_steps=50,
    seed=42
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics
)

## 7. Treinar
> **Observação**: esta célula pode levar minutos ou horas dependendo do tamanho do dataset e do hardware. Para testes rápidos, reduza `num_train_epochs` ou use amostra menor.

In [ ]:
# Descomente para treinar
# trainer.train()

## 8. Avaliar e interpretar resultados
Depois do treinamento, avaliamos o modelo no conjunto de validação e examinamos a matriz de confusão.

In [ ]:
# metrics = trainer.evaluate()
# print(metrics)

# from sklearn.metrics import confusion_matrix, classification_report
# import torch
# preds = trainer.predict(val_ds).predictions
# y_pred = np.argmax(preds, axis=-1)
# y_true = val_ds['labels']

# print(confusion_matrix(y_true, y_pred))
# print(classification_report(y_true, y_pred, digits=3))

## 9. Fazer previsões em novos casos
Função helper que devolve `pred` (0 ou 1) e as probabilidades para cada classe.

In [ ]:
def prever(texto_manifestacao, texto_resposta):
    seq = texto_manifestacao + ' [SEP] ' + texto_resposta
    tokens = tokenizer(seq, return_tensors='pt', truncation=True, padding='max_length', max_length=256)
    with torch.no_grad():
        logits = model(**tokens).logits
    prob = torch.softmax(logits, dim=1).squeeze()
    pred = torch.argmax(prob).item()
    return pred, prob.tolist()

# Exemplo de uso (modelo deve estar treinado!)
# pred, prob = prever(
#     'Cliente reclama de atraso na entrega.',
#     'Entramos em contato com o cliente por telefone em 07/05 às 14h e confirmamos nova data de entrega para 10/05.'
# )
# print(f'Previsão: {pred}  Probabilidades: {prob}')

## 10. Salvar e recarregar o modelo
```python
trainer.save_model('bertimbau_resposta_final')
```
Depois, em outro script ou notebook, basta:
```python
model = AutoModelForSequenceClassification.from_pretrained('bertimbau_resposta_final')
tokenizer = AutoTokenizer.from_pretrained('bertimbau_resposta_final')
```

## Observações finais
* Use GPU (`model.to('cuda')` ou `device_map='auto'`) para acelerar o treinamento.
* Ajuste `max_length` se suas respostas forem muito longas.
* Se a classe 1 ou 0 for rara, experimente reamostrar ou usar pesos de classe.
* Ferramentas como **LIME** ou **SHAP** podem ajudar a explicar as previsões do modelo.